In [42]:
from datasets import load_dataset

In [43]:
dataset = load_dataset('ms_marco', 'v1.1')

In [44]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 10047
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 82326
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 9650
    })
})

In [45]:
dataset = load_dataset('ms_marco', 'v1.1', split='train')

In [46]:
dataset

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 82326
})

In [47]:
type(dataset)

datasets.arrow_dataset.Dataset

In [48]:
def preprocess(example):
    positive_passages=[]
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==1:
            positive_passages.append(example["passages"]["passage_text"][p[0]])
    negative_passages = []
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==0:
            negative_passages.append(example["passages"]["passage_text"][p[0]])
    if (len(positive_passages)>0 and len(negative_passages)>=5):
        positive = positive_passages[0]
        negatives = negative_passages[:5]
        return {
            "query": example["query"],
            "positive": positive,
            "negatives": negatives
        }
    else:
        return {"query": None, "positive": None, "negatives": None}

In [49]:
processed_data = dataset.map(preprocess, remove_columns=dataset.column_names)

In [50]:
processed_data = processed_data.filter(lambda x: x['query'] is not None and x['positive'] is not None)

In [51]:
contrastive_pairs = []
for item in processed_data:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [52]:
len(contrastive_pairs)

74538

In [53]:
from torch.utils.data import DataLoader

In [54]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [55]:
contrastive_dataset = ContrastiveDataset(contrastive_pairs[0:4])

In [56]:
data_loader = DataLoader(contrastive_dataset, batch_size=2, shuffle=True)

In [57]:
len(data_loader)

2

In [58]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [59]:
model = AutoModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [60]:
from peft import LoraConfig, get_peft_model

In [61]:
lora_config = LoraConfig(
    task_type= "FEATURE_EXTRACTION"
)

In [62]:
lora_model = get_peft_model(model, lora_config)

In [63]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 109,777,152 || trainable%: 0.2686


In [64]:
import torch

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [66]:
import torch.optim as optim
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=5e-5)

In [67]:
num_epochs=2

In [68]:
import torch.nn.functional as F

In [76]:
def compute_laplacian(similarity_matrix):
    similarity_matrix = similarity_matrix.float()
    degree_matrix = torch.diag(torch.sum(similarity_matrix, dim=1))
    laplacian_matrix = degree_matrix - similarity_matrix

    return laplacian_matrix

In [77]:
def harmonic_distance(laplacian_matrix):
    laplacian_pseudo_inv = torch.pinverse(laplacian_matrix)
    harmonic_distances = []
    for anchor_idx in range(0, len(laplacian_matrix), 7):
        anchor_node=torch.zeros(len(laplacian_matrix), dtype=torch.float32)
        anchor_node[anchor_idx] = 1

        distances = []
        for i in range(7-1):
            node=torch.zeros(len(laplacian_matrix), dtype=torch.float32)
            node[anchor_idx+(i+1)] = 1
            diff = anchor_node - node
            dist = torch.matmul(torch.matmul(diff.T, laplacian_pseudo_inv), diff)
            distances.append(dist)
        
        harmonic_distances.append(torch.stack(distances))
    return(harmonic_distances)

In [78]:
def harmonic_loss(distances):
    logits=-torch.stack(distances, dim=0)
    print(logits)
    labels = torch.zeros(logits.size(0), dtype=torch.long, device=logits.device)
    loss = torch.nn.CrossEntropyLoss()(logits, labels)
    return loss

In [72]:
dataset_val = load_dataset('ms_marco', 'v1.1', split='validation')
processed_data_val = dataset_val.map(preprocess, remove_columns=dataset_val.column_names)
processed_data_val = processed_data_val.filter(lambda x: x['query'] is not None and x['positive'] is not None)
contrastive_pairs_val = []
for item in processed_data_val:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_val.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })
contrastive_dataset_val = ContrastiveDataset(contrastive_pairs_val[0:4])
data_loader_val = DataLoader(contrastive_dataset_val, batch_size=2, shuffle=True)
len(data_loader_val)

2

In [73]:
def evaluate_mrr(model, data_loader_val):
    model.eval()

    total_rr = 0.0
    num_queries = 0

    with torch.no_grad(): 
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

            anchor_embedding = model(**anchor_input).last_hidden_state[:, 0, :]
            positive_embedding = model(**positive_input).last_hidden_state[:, 0, :]
            negative_embedding = [model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :] for neg in negative_texts]
            batch_embeddings = []
        
            for i in range(len(anchor_embedding)): 
                sample_embeddings = torch.cat(
                    [anchor_embedding[i:i+1],  
                    positive_embedding[i:i+1],  
                    torch.stack([neg[i] for neg in negative_embedding])],  
                    dim=0
                )
                batch_embeddings.append(sample_embeddings)

            all_embeddings = torch.cat(batch_embeddings, dim=0)
            sim=F.cosine_similarity(all_embeddings.unsqueeze(1), all_embeddings.unsqueeze(0), dim=2)
            laplacian = compute_laplacian(sim)
            distances=harmonic_distance(laplacian)
            all_similarities=-torch.stack(distances, dim=0)

            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()  # Reciprocal rank
            num_queries += len(positive_rank)

    mrr = total_rr / num_queries
    return mrr



In [79]:
for epoch in range(num_epochs):
    lora_model.train() 
    
    total_loss = 0.0
    for batch in data_loader:
       
        anchor_texts = batch[0]
        positive_texts = batch[1]
        negative_texts = batch[2]
     
        anchor_inputs = tokenizer(anchor_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        positive_inputs = tokenizer(positive_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
        anchor_embedding = lora_model(**anchor_inputs).last_hidden_state[:, 0, :]
        positive_embedding = lora_model(**positive_inputs).last_hidden_state[:, 0, :]
        negative_embedding = [lora_model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :] for neg in negative_texts]

        batch_embeddings = []

        for i in range(len(anchor_embedding)):
            sample_embeddings = torch.cat(
                [anchor_embedding[i:i+1],  
                positive_embedding[i:i+1],  
                torch.stack([neg[i] for neg in negative_embedding])], 
                dim=0
            )
            batch_embeddings.append(sample_embeddings)

        all_embeddings = torch.cat(batch_embeddings, dim=0)
        sim=F.cosine_similarity(all_embeddings.unsqueeze(1), all_embeddings.unsqueeze(0), dim=2)
        laplacian = compute_laplacian(sim)
        distances=harmonic_distance(laplacian)

        loss = harmonic_loss(distances)
        
        optimizer.zero_grad()  
        loss.backward()  
        optimizer.step()  
        
        total_loss += loss.item()
    print(total_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader)}")
    mrr_validation = evaluate_mrr(lora_model, data_loader_val)
    mrr_train = evaluate_mrr(lora_model, data_loader)
    print(f"Mean Reciprocal Rank (MRR) for training set: {mrr_train:.4f}")
    print(f"Mean Reciprocal Rank (MRR) for validation set: {mrr_validation:.4f}")

tensor([[-0.2188, -0.2344, -0.2188, -0.2188, -0.2344, -0.2188],
        [-0.2500, -0.2031, -0.2500, -0.2500, -0.2188, -0.2969]],
       grad_fn=<NegBackward0>)
tensor([[-0.2500, -0.2500, -0.2500, -0.2500, -0.2500, -0.0000],
        [-0.2500, -0.2500, -0.0000, -0.2500, -0.2500, -0.0000]],
       grad_fn=<NegBackward0>)
3.6521031856536865
Epoch 1/2, Loss: 1.8260515928268433
Mean Reciprocal Rank (MRR) for training set: 1.0000
Mean Reciprocal Rank (MRR) for validation set: 0.8000
tensor([[-0.2031, -0.2344, -0.2188, -0.2188, -0.2344, -0.2031],
        [-0.2344, -0.2500, -0.2500, -0.2344, -0.2344, -0.2969]],
       grad_fn=<NegBackward0>)
tensor([[-0.2188, -0.2500, -0.2344, -0.2969, -0.2188, -0.2344],
        [-0.2344, -0.2344, -0.2188, -0.2344, -0.2500, -0.2344]],
       grad_fn=<NegBackward0>)
3.556533098220825
Epoch 2/2, Loss: 1.7782665491104126
Mean Reciprocal Rank (MRR) for training set: 0.8750
Mean Reciprocal Rank (MRR) for validation set: 0.5083
